Notebook for training a diffusion model on mnist data

Sources:

[The Network Code](https://github.com/FluxML/model-zoo/blob/master/vision/diffusion_mnist/diffusion_mnist.jl)

[The Sampling Code](https://github.com/FluxML/model-zoo/blob/master/vision/diffusion_mnist/diffusion_plot.jl)

[Useful Explainer](https://yang-song.net/blog/2021/score/)

[Original Paper](https://arxiv.org/pdf/2011.13456.pdf)

TO DO:
- Optimize code: training is not as fast as expected? 5min for 10 epochs with 1000 images
    - Mainly due to compiling. After that about 18min for 60k images and 30 epochs
- Make compatible with job submission
- Include more samples: Predictor Corrector and Diff. Eq. solver (see explainer and sampling code links)
- Image inpainting: adapt sampling to image inpainting (see python code attached to original paper)

In [1]:
# Dependencies
using Pkg
Pkg.activate("/home/mverlaan/bathy_machine_learning/julia_ml_tests.jl.git/")
# include("diffusion_unet_model.jl")
include("unet_large.jl")
# include("diffusion_sampling.jl")
using MLDatasets
using CUDA
using ProgressMeter: Progress, next!
using BSON

  Activating project at `~/bathy_machine_learning/julia_ml_tests.jl.git`


In [8]:
# Hyper Parameters
n_images = 60000
channels = [32, 64, 128, 256] # No of channels in Unet conv layers
embed_dim = 256 # dimensionality of Fourier projection
scale = 30.0f0 # scale parameter of Fourier projection
lr = 0.0001

nr_epochs = 50
batch_size = 32
device = gpu

gpu (generic function with 5 methods)

In [9]:
# Get Train data
xtrain, ytrain = MNIST(:train)[1:n_images]
xtrain = reshape(xtrain, (28,28,1,:));

In [10]:
# Things for the training loop

data_loader = Flux.DataLoader((xtrain,ytrain), batchsize=batch_size, shuffle=true) |> device
# if device == gpu
#     data_loader |> device
# end
# data_loader

1875-element DataLoader(::MLUtils.MappedData{:auto, typeof(gpu), Tuple{Array{Float32, 4}, Vector{Int64}}}, shuffle=true, batchsize=32)
  with first element:
  (28×28×1×32 CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}, 32-element CuArray{Int64, 1, CUDA.Mem.DeviceBuffer},)

In [5]:
unet_model = UNET(channels, embed_dim, scale) |> device

opt = Adam(lr)

params = Flux.params(unet_model);

In [6]:
# Test loss function

test_loss = score_loss(unet_model, first(data_loader)[1])

1113.8665f0

In [11]:
# Training loop

progress = Progress(length(1:nr_epochs))
for epoch in 1:nr_epochs
    println("Starting epoch no. $(epoch)")

    batch_losses = zeros(length(data_loader))
    batch_no = 1

    for (x,_) in data_loader
        loss,grad = Flux.withgradient(params) do 
            score_loss(unet_model,x)            
        end
        Flux.Optimise.update!(opt,params,grad)
        batch_losses[batch_no] = loss
        batch_no += 1
    end
    next!(progress, showvalues=[(:loss, mean(batch_losses))])
end

Starting epoch no. 1


Starting epoch no. 2


Progress:   4%|█▋                                       |  ETA: 0:40:10


Starting epoch no. 3



Progress:   6%|██▌                                      |  ETA: 0:39:25


...


Progress:  96%|███████████████████████████████████████▍ |  ETA: 0:01:40


Starting epoch no. 49



Progress:  98%|████████████████████████████████████████▏|  ETA: 0:00:50


Starting epoch no. 50



Progress: 100%|█████████████████████████████████████████| Time: 0:41:39
  loss:  14.226620218658447

In [ ]:
unet_model

In [ ]:
# Model back to cpu

unet_cpu = unet_model |> cpu

In [ ]:
test_sample = first(data_loader)[1] |> cpu

In [ ]:
test_loss = score_loss(unet_cpu, test_sample)

In [12]:
model = unet_model |> cpu
save_path = joinpath("models","diffusion_fullunet_mnist60k_epoch50.bson")

"models/diffusion_fullunet_mnist60k_epoch50.bson"

In [13]:
# BSON.@save save_path model